In [211]:
import pandas as pd
df=pd.read_csv("UNSW_NB15_training-set.csv")

In [212]:
df.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [213]:
print(df.shape[1])

45


In [214]:
from sklearn.preprocessing import LabelEncoder

In [215]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Initialize LabelEncoder
le = LabelEncoder()

# Fit on attack_cat column
le.fit(df['attack_cat'])

# Get all classes
classes = list(le.classes_)

# Move 'Normal' to index 0
if 'Normal' in classes:
    classes.remove('Normal')
    classes = ['Normal'] + classes  # Normal first

# Create a mapping dictionary
label_mapping = {label: idx for idx, label in enumerate(classes)}

# Map attack_cat to numerical labels
df['label'] = df['attack_cat'].map(label_mapping)

print(label_mapping)
print(df[['attack_cat', 'label']].head())


{'Normal': 0, 'Analysis': 1, 'Backdoor': 2, 'DoS': 3, 'Exploits': 4, 'Fuzzers': 5, 'Generic': 6, 'Reconnaissance': 7, 'Shellcode': 8, 'Worms': 9}
  attack_cat  label
0     Normal      0
1     Normal      0
2     Normal      0
3     Normal      0
4     Normal      0


In [216]:
#save to CSV
df.to_csv("dataset_label.csv", index=False)

In [217]:
df.drop('attack_cat', axis=1, inplace=True)


In [218]:
# Drop id
df.drop("id",axis=1,inplace=True)

In [219]:
df.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,...,1,1,1,0,0,0,1,1,0,0
1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,...,1,1,2,0,0,0,1,6,0,0
2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,...,1,1,3,0,0,0,2,6,0,0
3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,...,1,1,3,1,1,0,2,1,0,0
4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,...,2,1,40,0,0,0,2,39,0,0


In [220]:
df.drop_duplicates(inplace=True)


In [221]:
#check for dupilicates
print("Duplicates:",df.duplicated().sum())

Duplicates: 0


In [222]:
df = df.sample(frac=1, random_state=42)
df.reset_index(drop=True, inplace=True)


In [223]:
#check for dupilicates
print("Duplicates:",df.duplicated().sum())

Duplicates: 0


In [224]:
#check for categorical columns
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
print("Categorical columns (by dtype):", categorical_columns.tolist())

Categorical columns (by dtype): ['proto', 'service', 'state']


In [83]:
import joblib

In [225]:
X = df.drop('label', axis=1)
y = df['label']


In [226]:

duplicates_mask = X.duplicated(keep='first')


In [227]:
X = X[~duplicates_mask].reset_index(drop=True)


In [228]:
y = y[~duplicates_mask].reset_index(drop=True)


In [229]:
#check for dupilicates
print("Duplicates:",X.duplicated().sum())

Duplicates: 0


In [230]:
# Categorical columns
categorical_cols = ['proto', 'service', 'state']

# Dictionary to store encoders
encoders = {}

# Apply label encoding and save mappings
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    encoders[col] = le  # store encoder for later use

# Save encoders for future use
#joblib.dump(encoders, 'label_encoders.pkl')



In [231]:
#check for dupilicates
print("Duplicates:",X.duplicated().sum())

Duplicates: 0


In [232]:
print(X.shape[1])

42


In [233]:
import joblib
joblib.dump(encoders,'encoders.pkl')



['encoders.pkl']

In [234]:
#check for dupilicates
print("Duplicates:",X.duplicated().sum())

Duplicates: 0


In [236]:
#check for infinite values
import numpy as np

# Select only numeric columns
X_numeric = X.select_dtypes(include=np.number)

# Check for infinite values in numeric columns
print(np.isinf(X_numeric).sum())

dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports      0
dtype: int64


In [237]:
#conert infinite values into nan
#X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [238]:
#check for null values
#print(df.isnull().sum())
#print(df.isnull().any())
print("Total nulls:", X.isnull().values.sum())


Total nulls: 0


In [239]:
#check for dupilicates
print("Duplicates:",X.duplicated().sum())

Duplicates: 0


In [242]:
# Check for placeholder or empty-string values in X
for col in X.columns:
    unique_vals = X[col].unique()
    if '-' in unique_vals or '' in unique_vals:
        print(f"Column {col} has '-' or empty values")


In [243]:
#check for dupilicates
print("Train duplicates:", X.duplicated().sum())

Train duplicates: 0


In [244]:
#feature scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

X_train = scaler.fit_transform(X)

In [246]:
#check for dupilicates
print("Train duplicates:", X.duplicated().sum())

Train duplicates: 0


In [247]:
print(X.shape[1])

42


In [248]:
print(X_train.shape[1])

42


In [249]:
# Get column names
feature_names = X.columns

# Create DataFrames from the scaled numpy arrays
X_train_df = pd.DataFrame(X_train, columns=feature_names)

# Concatenate the DataFrames with the target Series
train_df = pd.concat([X_train_df, y.reset_index(drop=True)], axis=1)

train_df.to_csv("pretrain_dataset.csv", index=False)


In [250]:
#check for dupilicates
print("Train duplicates:", train_df.duplicated().sum())

Train duplicates: 0


In [251]:
train_df.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
0,1.250022e-02,0.856061,0.000000,0.250,0.000936,0.000729,0.000041,0.000024,0.000023,0.996078,...,0.00,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,7
1,2.828167e-04,0.856061,0.000000,0.250,0.004888,0.004556,0.000226,0.002255,0.005716,0.121569,...,0.00,0.0,0.000,0.0,0.0,0.000000,0.016949,0.081967,0.0,0
2,4.236651e-03,0.856061,0.416667,0.250,0.000936,0.000911,0.000057,0.000401,0.000075,0.243137,...,0.00,0.0,0.000,0.0,0.0,0.033333,0.000000,0.000000,0.0,4
3,1.666667e-07,0.901515,0.000000,0.375,0.000104,0.000000,0.000097,0.000000,0.100000,0.996078,...,0.00,0.0,0.000,0.0,0.0,0.000000,0.033898,0.000000,0.0,6
4,7.365401e-03,0.856061,0.000000,0.000,0.000520,0.000182,0.000074,0.000006,0.000016,0.243137,...,0.06,0.0,0.125,0.0,0.0,0.000000,0.067797,0.131148,0.0,0


In [252]:
y.value_counts()

label
0    51779
4    18719
5    14943
7     6450
6     3949
3     2741
8     1088
1      632
2      618
9      121
Name: count, dtype: int64

In [253]:
import pickle
import joblib

# Save the objects
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
 
#joblib.dump(encoders,'encoders.pkl')


In [254]:
print(encoders)

{'proto': LabelEncoder(), 'service': LabelEncoder(), 'state': LabelEncoder()}


In [255]:
print(le)

LabelEncoder()
